# Assignment Applied Machine Learning BSc IK 

## Notebook made by

**Gebruik graag dit formaat**

* Voor de namen:  voornaam rest van je naam, voornaam rest van je naam,....
* je studentnummers: hetzelfde: scheidt met `,`
* je emails: hetzelfde: scheidt met `,`
* voor je groep: **alleen de hoofdletter** (iets als  `A` of `B` dus)

__Namen__:Anoniem
__Emails__:Anoniem
__Student id__:Anoniem
__Groep__:Anoniem

## Toelichting

* Een aantal opgaven worden automatisch nagekeken. Bij vrijwel alle opdrachten staan er een paar tests onder de opdracht, dit is voornamelijk om te zorgen dat je de juiste type output geeft. Dit zijn dus *NIET* alle tests, die komen er bij het graden nog bij.
* Elke vraag is 1 punt waard, tenzij anders aangegeven. Soms is die punt onderverdeeld in deelpunten, maar niet altijd. 

## Voor het inleveren!

* Pas niet de cellen aan, vooral niet die je niet kunt editen. Dit levert problemen op bij nakijken. Twijfel je of je per ongeluk iets hebt gewijzigd, kopieer dan bij inleveren je antwoorden naar een nieuw bestand, zodat het niet fout kan gaan.

* Zorg dat de code goed runt van boven naar beneden, verifieer dat door boven in Kernel -> Restart & Run All uit te voeren

## Na het inleveren!

* Het gebeurt erg vaak dat mensen een "leeg bestand" inleveren. Vaak een andere versie van de opgave die nog ergens op je computer rondslingerde. Zonde van al je werk toch!
* Dus, lever **minstens een half uur voor tijd in**. Download dan wat je hebt ingeleverd op Canvas. Geef het een andere naam om verwarring te voorkomen. En draai alle cellen, en bekijk het. Geen syntax fouten? Alle vragen gemaakt? Dan zit het vast wel goed, en hoef je niet in de zenuwen te zitten.

# Applied Machine Learning W2

This assignment tests the concepts that have been discussed in this week's lecture, and is divided in several (mostly) independent exercises. 

1. [Agglomerative Clustering](#agglo)
2. [DBSCAN](#dbscan)
3. [Evaluation](#elm)
4. [Data Normalization](#normalization)
5. [PCA](#pca)

### TIP
The second assignment, which asks you to implement the DBSCAN algorithm, can be a bit challenging, so don't forget about the other exercises! Just start working on another part of the assignment if you get stuck, and come back later.

In [ ]:
%matplotlib inline

import os
import sys

import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from nose.tools import assert_count_equal, assert_equal
from numpy.testing import *
from sklearn import datasets
# Please do not remove this: 
np.random.seed(31415)

<a id="agglo">

# 1 Agglomerative Clustering

1. We defined the Ward criterion in class. Now you define from scratch, similar as we did with Ward, *average* and *complete*.
2. Create the complete algrithm for bottom up agglomerative clustering from scratch, giving, just like in sklearn, the possibility to choose the linking function. We will help you by having you implement the parts needed as seperate functions.
3. Use the Euclidean distance in your functions.
3. Try out your code on the iris dataset.
4. Compare it to sklearn. I mean the outcomes on the same input. 
5. Use the technique described in the book to create a dendogram.

## Tips
* explain what you are doing
* test and test and test
* refactor, refactor
* Be proud. Don't despair if it takes hours to write a few lines of code.
    * That is because Python is such a fantastic high level language.

In [ ]:

import numpy as np

def average_linkage(cluster1, cluster2) -> np.float64:
    n1, n2 = len(cluster1), len(cluster2)
    sum_distances = 0
    for i in range(n1):
        for j in range(n2):
            sum_distances += np.linalg.norm(cluster1[i] - cluster2[j])
    distance = sum_distances / (n1 * n2)
    return distance

def complete_linkage(cluster1, cluster2) -> np.float64:
 
    n1, n2 = len(cluster1), len(cluster2)
    max_distance = 0
    for i in range(n1):
        for j in range(n2):
            distance = np.linalg.norm(cluster1[i] - cluster2[j])
            if distance > max_distance:
                max_distance = distance
    return max_distance




In [ ]:
# check that you get the right output
cluster_1 = np.random.rand(5, 2)
cluster_2 = np.random.rand(10, 2)
assert_equal(type(average_linkage(cluster_1, cluster_2)), np.float64)

After implementing the two linkage functions, you will implement a single step of the clustering algorithm, which will return the IDs of the clusters in the input that should be merged. As input you will have both the original input data, as well as a dictionary of clusters, where each key is an ID, and each value is a list of points belonging to that cluster. You can use fancy indexing from numpy to get the rows belonging to a cluster easily:

```
# suppose we have input data X
clusters = {0: [0, 1, 2, 3], 4: [4, 6, 7]}
# get the vectors of our first cluster
cluster_vectors = X[clusters[0]]
```

In [ ]:
def clustering_step(input_data, cluster_dict, linkage_type: str) -> tuple:
    
    assert linkage_type in ['average', 'complete']
    cluster_ids = [None, None]
    min_distance = float('inf')
    
    linkage_function = average_linkage if linkage_type == 'average' else complete_linkage
    
  
    for id1 in cluster_dict:
        for id2 in cluster_dict:
            if id1 != id2:
        
                cluster1_vectors = input_data[cluster_dict[id1]]
                cluster2_vectors = input_data[cluster_dict[id2]]
                distance = linkage_function(cluster1_vectors, cluster2_vectors)
                
               
                if distance < min_distance:
                    min_distance = distance
                    cluster_ids = [id1, id2]
    
    return tuple(sorted(cluster_ids))


In [ ]:
input_array = np.array([
    [0, 0, 0],
    [1, 1, 1],
    [3, 3, 3]])
clusters = {0: [0], 1: [1], 2: [2]}

assert_equal(type(clustering_step(input_array, clusters, linkage_type='average')), tuple)

In our last step, we will implement the complete algorithm for agglomerative clustering, using our previously created `clustering_step` and linkage functions. For the return value, return a list of labels where each label corresponds to the class of the instance. It doesn't matter if these values are not 0/1 as long as items in the same cluster share the same label. 

**TIP**
You can delete key value pairs from dictionaries using the `del dictionary[key]` syntax

In [ ]:
# in our version, we will define a number of clusters we want to find
# you may also define this in sklearn for your comparison later
def agglomerative_clustering(input_points, number_of_clusters: int, linkage_type: str):
    assert linkage_type in ['average', 'complete']
    

    clusters = {i: [i] for i in range(len(input_points))}

    # Look at the stopping condition, understand why?
    while len(clusters) != number_of_clusters:
        # Find the two clusters to merge
        cluster_ids_to_merge = clustering_step(input_points, clusters, linkage_type)
        
        # Merge the clusters
        merged_cluster = clusters[cluster_ids_to_merge[0]] + clusters[cluster_ids_to_merge[1]]
        
        # Remove old clusters and add the new merged cluster
        del clusters[cluster_ids_to_merge[0]]
        del clusters[cluster_ids_to_merge[1]]
        clusters[min(cluster_ids_to_merge)] = merged_cluster
    
    # create the output array (we will do this for you)
    output_labels = np.zeros((input_points.shape[0], 1))
    for i, point_ids in enumerate(clusters.values()):
        output_labels[point_ids] = i
    
    return output_labels


In [ ]:
# Did we actually create the right amount of clusters?
samples = np.random.rand(10, 3)
labels = agglomerative_clustering(samples, number_of_clusters=4, linkage_type="average")
assert_equal(len(np.unique(labels)), 4)

In [ ]:
from sklearn.datasets import load_iris
from sklearn.cluster import AgglomerativeClustering

# Loaddataset
data = load_iris()
X = data.data

number_of_clusters = 3
linkage_type = 'average'

custom_labels = agglomerative_clustering(X, number_of_clusters, linkage_type)

sklearn_model = AgglomerativeClustering(n_clusters=number_of_clusters, linkage=linkage_type)
sklearn_labels = sklearn_model.fit_predict(X)


print("Custom agglomerative clustering labels:")
print(custom_labels.flatten())

print("Sklearn agglomerative clustering labels:")
print(sklearn_labels)






Now that you have created the agglomerative algorithm, create the dendogram and plot it.

In [ ]:

def agglomerative_clustering_dendrogram(input_points, linkage_type: str):
    assert linkage_type in ['average', 'complete']
    
    clusters = {i: [i] for i in range(len(input_points))}
    merge_steps = []
    new_cluster_id = len(input_points)

    while len(clusters) != 1:
        cluster_ids_to_merge = clustering_step(input_points, clusters, linkage_type)
        
        # Calculate the distance between the clusters to be merged
        cluster1_vectors = input_points[clusters[cluster_ids_to_merge[0]]]
        cluster2_vectors = input_points[clusters[cluster_ids_to_merge[1]]]
        linkage_function = average_linkage if linkage_type == 'average' else complete_linkage
        distance = linkage_function(cluster1_vectors, cluster2_vectors)

        
        merged_cluster = clusters[cluster_ids_to_merge[0]] + clusters[cluster_ids_to_merge[1]]
        merge_steps.append([cluster_ids_to_merge[0], cluster_ids_to_merge[1], distance, len(merged_cluster)])
        
 
        del clusters[cluster_ids_to_merge[0]]
        del clusters[cluster_ids_to_merge[1]]
        clusters[new_cluster_id] = merged_cluster
        new_cluster_id += 1
    
    return merge_steps


linkage_type = 'average'
dendrogram_data = agglomerative_clustering_dendrogram(X, linkage_type)

# Plot  dendrogram
plt.figure(figsize=(10, 5))
dendrogram(dendrogram_data)
plt.title("Dendrogram - Agglomerative Clustering")
plt.xlabel("Data points")
plt.ylabel("Distance")
plt.show()



<a id="dbscan" />

# 2 DBscan

In class and in section of the book 3.5.3 the DBScan algorithm was discussed.
In this assignment we are going to implement the DBScan metric from scratch, working from first principles and using euclidean distance as the distance metric.

**TIPS**
- Implementing this from scratch is a challenge, but it should be doable, make sure you really understand/can explain the algorithm before you start coding, this should already help you when programming. Like before, we will also make you implement sub functions first to help you along.
- Remember that in the algorithm we include the sample itself in the calculation of the distances and `min_samples`, you don't have to write separate logic to exlcude the point itself from the neighbour calculations.

In [ ]:


def get_neighbours(input_data, point_index, eps) -> list:
 
    neighbors = []
    point = input_data[point_index]

    for i, other_point in enumerate(input_data):
        distance = np.linalg.norm(point - other_point)
        if distance <= eps:
            neighbors.append(i)

    return neighbors


In [ ]:
# check that the output contains integers, which should correspond
# to the indices of the neighbours
test_data = np.random.rand(5, 3)
assert_equal(type(get_neighbours(test_data, 0, eps=1)[0]), int)

After we have implemented the function to get all neighbours of a point for a specific point, we will write a function that, starting from a specific point, finds all neighbour points, and their neighbours points, and so on, until the cluster can no longer be expanded.

## TIPS
 - In this function and the next, we will assume points that have a value of `-1` in the cluster are ure (currently) outliers, and points with a value of `-2` have not yet been examined.

In [ ]:
def expand(input_data, clusters, point, class_label, eps, min_samples):

    neighbours = get_neighbours(input_data, point, eps)
    clusters[point] = class_label

    i = 0
    while i < len(neighbours):
        neighbour_point = neighbours[i]
        point_cluster = clusters[neighbour_point]

        if point_cluster == -2:  # unassigned point
  
            clusters[neighbour_point] = class_label

            neighbour_neighbours = get_neighbours(input_data, neighbour_point, eps)

          
            if len(neighbour_neighbours) >= min_samples:
                neighbours += neighbour_neighbours

  
        elif point_cluster == -1:
            clusters[neighbour_point] = class_label

        i += 1

    return clusters


In the cell below we will put everything we made so far together and create the complete DBSCAN algorithm. 


In [ ]:
def dbscan(input_data, eps, min_samples):
    """
    Implements the complete dbscan algorithm using the helper functions we wrote before.
    Takes as input the complete input matrix, the maximum distance neighbours can be apart, eps,
    and the minimal number of samples that are neighbours from a point to let it be considered
    a core point. Returns an array with for each item the class label of that item, where -1
    indicates that the item is an outlier.
    """
    current_class = 0
    
    clusters = [-2 for _ in range(input_data.shape[0])]

    for i in range(0, input_data.shape[0]):
        if clusters[i] == -2:  # unassigned point
            neighbours = get_neighbours(input_data, i, eps)

         
            if len(neighbours) >= min_samples:
                clusters = expand(input_data, clusters, i, current_class, eps, min_samples)
                current_class += 1
            else:
                clusters[i] = -1  # outlier

    return np.array(clusters)


Compare the output of your algorithm to the output of sklearns `DBSCAN` in the cell below, explaing your tests in the comments of the cell

### TIP

- Have a look at `make_blobs` from the sklearn.datasets module, this is a nice function with which you can easily
create some sample clusters to test your algorithm.

In [ ]:


# data
n_samples = 300
centers = [(0, 0), (5, 5), (10, 10)]
X, y = make_blobs(n_samples=n_samples, centers=centers, cluster_std=1, random_state=42)

# iplementatie DBSCAN
clusters = dbscan(X, eps=2, min_samples=5)
print("mijn implementatie van DBSCAN:")
print(clusters)

# Sklearn DBSCAN
sklearn_dbscan = SklearnDBSCAN(eps=2, min_samples=5)
sklearn_clusters = sklearn_dbscan.fit_predict(X)
print("Sklearn's implementation van DBSCAN:")
print(sklearn_clusters)

# Visualise
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis')
ax[0].set_title("mijn implementatie van DBSCAN")

ax[1].scatter(X[:, 0], X[:, 1], c=sklearn_clusters, cmap='viridis')
ax[1].set_title("Sklearn's implementation van DBSCAN")

plt.show()


<a id="elm" />

# 3 Evaluation

We can evaluate a cluster algorithm against a gold standard labelling. The tricky part then is which cluster to assign to which label. Think of Iris, and you have clustered it into 3 groups. Then which group do you want to assign to which species?

A smart idea called BCubed, and later improved to [ELM](https://dl.acm.org/doi/abs/10.1145/3539813.3545121) can be used to evaluate the quality of a clustering.

* implement ELM
* For this exercise, implement the F1 score, where you calculate F1 for each element and then average it over all elements
* test it with a number of clusterings you made of the IRIs set and another own found set with gold standard.
 - If you managed to implement your own version DBSCAN you can use these to make clusters of the IRIS dataset, otherwise just use DBSCAN from `sklearn`.
* Experiment with normalization, and see the effect.
* You are not training anything, so you simply cluster the full set and use the same set as test set.

* Next to the code, you write a small report in a markdown cell on your findings.

**TIPS**:
 - The ELM algorithm operates on points, and compares the cluster a point is in in the
 gold standard with a cluster a point is in in the prediction. 
 - Using sets is useful!
 - Start by creating a nested dict where each key is the index of a point, and each value is a dict with the indices of its neighbours, exluding the element itself. Do this for buth the gold standard and the predicted cluster.


### Test dbscan's parameters

* Using our ELM algorithm, try several options for the DBSCAN algorithm and report which values worked best.

In [ ]:
def ELM(gold_standard_labels, predicted_labels):
    def create_cluster_dict(labels):
        cluster_dict = {}
        for i, label in enumerate(labels):
            if label not in cluster_dict:
                cluster_dict[label] = set()
            cluster_dict[label].add(i)
        return cluster_dict

    gold_standard_dict = create_cluster_dict(gold_standard_labels)
    predicted_dict = create_cluster_dict(predicted_labels)

    def calculate_F1(gold_standard, predicted):
        precision = len(gold_standard.intersection(predicted)) / len(predicted)
        recall = len(gold_standard.intersection(predicted)) / len(gold_standard)
        if precision + recall == 0:
            return 0
        return 2 * (precision * recall) / (precision + recall)

    num_points = len(gold_standard_labels)
    score = []

    for point in range(num_points):
        gold_standard_cluster = gold_standard_dict[gold_standard_labels[point]]
        predicted_cluster = predicted_dict[predicted_labels[point]]
        point_F1 = calculate_F1(gold_standard_cluster, predicted_cluster)
        score.append(point_F1)

    return np.mean(score)


In [ ]:
assert_equal(type(ELM([0, 1, 2, 2], [0, 0, 0, 0])), np.float64)

In [ ]:
# Try different variations of DBSCAN, and evaluate using the ELM score we just defined

iris = load_iris()
X, y = iris.data, iris.target

#normaliseren
X_normalized = StandardScaler().fit_transform(X)

parameter_combinations = [
    (eps, min_samples)
    for eps in np.arange(0.1, 1.1, 0.1)
    for min_samples in range(1, 11)
]

best_params = None
best_elm_score = -1

for eps, min_samples in parameter_combinations:
    dbscan = SklearnDBSCAN(eps=eps, min_samples=min_samples)
    predicted_labels = dbscan.fit_predict(X_normalized)
    elm_score = ELM(y, predicted_labels)
    
    if elm_score > best_elm_score:
        best_elm_score = elm_score
        best_params = (eps, min_samples)

    print(f"eps: {eps}, min_samples: {min_samples}, ELM score: {elm_score}")

print(f"Best parameters: eps={best_params[0]}, min_samples={best_params[1]}, ELM score: {best_elm_score}")


<a id="normalization" />

# Preprocessing and scaling section 3.2


1. Download iris data, and Z-transform it using your own code.
    2. Give a convincing check/test(s) that it worked.
2. Similary for the `min-max` scalar and sklearn's `normalizer`.
    * Of course you also describe in good English and with a math formula what this scaler is doing.
3. Figure out how sklearn's *RobustScaler* works, and implement it using your own code. You can use numpy to get the 1st and 3rd quartile.
4. Test all your code on iris and at least one other dataset.
    * Make your own tests which really test the math.
    * Run the same ytransformation in sklearn and use something like `assert_array_almost_equal` to test that you are close enough to sklearn's.

Answer both in code and in a markdown cell.

In [ ]:
iris_X, iris_y = sklearn.datasets.load_iris(return_X_y=True)

def z_transform(input_data):
    mean = np.mean(input_data, axis=0)
    std = np.std(input_data, axis=0)
    output_data = (input_data - mean) / std
    return output_data


def my_normalize(input_data):
    norms = np.linalg.norm(input_data, axis=1)[:, np.newaxis]
    output_data = input_data / norms
    return output_data

    
def min_max(input_data, min_val=0, max_val=1):
    min_input = np.min(input_data, axis=0)
    max_input = np.max(input_data, axis=0)
    output_data = (input_data - min_input) / (max_input - min_input) * (max_val - min_val) + min_val
    return output_data

def robust_scaler(input_data):
    medians = np.median(input_data, axis=0)
    q1, q3 = np.percentile(input_data, [25, 75], axis=0)
    iqr = q3 - q1
    output_data = (input_data - medians) / iqr
    return output_data





# Z-transform
z_transformed = z_transform(iris_X)
sklearn_z_transformed = StandardScaler().fit_transform(iris_X)
assert_array_almost_equal(z_transformed, sklearn_z_transformed)

# Min-max scaling
min_max_transformed = min_max(iris_X)
sklearn_min_max_transformed = MinMaxScaler().fit_transform(iris_X)
assert_array_almost_equal(min_max_transformed, sklearn_min_max_transformed)

# Normalization
normalized = my_normalize(iris_X)
sklearn_normalized = Normalizer().fit_transform(iris_X)
assert_array_almost_equal(normalized, sklearn_normalized)

# Robust scaling
robust_scaled = robust_scaler(iris_X)
sklearn_robust_scaled = RobustScaler().fit_transform(iris_X)
assert_array_almost_equal(robust_scaled, sklearn_robust_scaled)

# Original iris data
plt.scatter(iris_X[:, 0], iris_X[:, 1], c=iris_y)
plt.title("Original iris data")
plt.xlabel("Sepal length")
plt.ylabel("Sepal width")
plt.show()

# Z-transformed iris data
plt.scatter(z_transformed[:, 0], z_transformed[:, 1], c=iris_y)
plt.title("Z-transformed iris data")
plt.xlabel("Sepal length (z-transformed)")
plt.ylabel("Sepal width (z-transformed)")
plt.show()

# Min-max scaled iris data
plt.scatter(min_max_transformed[:, 0], min_max_transformed[:, 1], c=iris_y)
plt.title("Min-max scaled iris data")
plt.xlabel("Sepal length (min-max scaled)")
plt.ylabel("Sepal width (min-max scaled)")
plt.show()

# Normalized iris data
plt.scatter(normalized[:, 0], normalized[:, 1], c=iris_y)
plt.title("Normalized iris data")
plt.xlabel("Sepal length (normalized)")
plt.ylabel("Sepal width (normalized)")
plt.show()

# Robust scaled iris data
plt.scatter(robust_scaled[:, 0], robust_scaled[:, 1], c=iris_y)
plt.title("Robust scaled iris data")
plt.xlabel("Sepal length (robust scaled)")
plt.ylabel("Sepal width (robust scaled)")
plt.show()


Briefly describe for each scaler what they do (and give the formula)

Z-score or StandardScaler: This scaler adjusts the data's mean and standard deviation to 0 and 1, respectively. The calculation is (x - mean) / std, where x is the data point, mean the data mean, and std the data standard deviation.
This scaler, known as a min-max scaler, adjusts the data so that it is between a defined minimum and maximum value. The equation is (x - min) / (max - min) * (max_val - min_val) + min_val, where x is the data point, min and max are the data's minimum and maximum values, and min_val and max_val are the user-specified minimum and maximum values.
Normalization, is the process of scaling data so that each data point has a unit norm. The formula is x / ||x||, where ||x|| is the L2-norm of x and x is the data point.
robust scaler: This scaler increases the data's resistance to outliers by scalating it using the interquartile range (IQR) rather than the standard deviation. (x - median) / IQR is the formula, where x is the data point, median is the data median, and IQR is the data's interquartile range.

For each of the functions you implemented above, write tests in the cell below to make sure your scaling is correct

In [ ]:


# Z-transform
z_transformed = z_transform(iris_X)
sklearn_z_transformed = StandardScaler().fit_transform(iris_X)
assert_array_almost_equal(z_transformed, sklearn_z_transformed)
plt.scatter(z_transformed[:, 0], z_transformed[:, 1], c=iris_y)
plt.title("Z-transformed Iris Dataset")
plt.show()

# Min-max scaling
min_max_transformed = min_max(iris_X)
sklearn_min_max_transformed = MinMaxScaler().fit_transform(iris_X)
assert_array_almost_equal(min_max_transformed, sklearn_min_max_transformed)
plt.scatter(min_max_transformed[:, 0], min_max_transformed[:, 1], c=iris_y)
plt.title("Min-max Scaled Iris Dataset")
plt.show()

# Normalization
normalized = my_normalize(iris_X)
sklearn_normalized = Normalizer().fit_transform(iris_X)
assert_array_almost_equal(normalized, sklearn_normalized)
plt.scatter(normalized[:, 0], normalized[:, 1], c=iris_y)
plt.title("Normalized Iris Dataset")
plt.show()

# Robust scaling
robust_scaled = robust_scaler(iris_X)
sklearn_robust_scaled = RobustScaler().fit_transform(iris_X)
assert_array_almost_equal(robust_scaled, sklearn_robust_scaled)
plt.scatter(robust_scaled[:, 0], robust_scaled[:, 1], c=iris_y)
plt.title("Robustly Scaled Iris Dataset")
plt.show()



<a id="pca" />

# PCA

### inspect
* get the sklearn digit dataset.
* transform it to 2 dimensions using PCA, you are allowed to use the `sklearn` function for this exercise.
* plot it in 2 dimensions using seaborn and use the `hue` parameter to see how much the digits are separated.
* Does it make sense? Do you see similar "confusions/overlap" as seen before in the book between certain digits?
 - Play around with the color palete to ensure that the different classes have clearly different colours.


### cluster

Run K-means on this reduced data (2 instead of 64 dimensions!), also run K-means on the original data. Then compare the two using the ELM-metric. For this experiment you are allowed to use the K means algorithm from sklearn.

In [ ]:
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

digits_X, digits_y = load_digits(return_X_y=True)


pca = PCA(n_components=2)
digits_pca = pca.fit_transform(digits_X)


sns.scatterplot(x=digits_pca[:,0], y=digits_pca[:,1], hue=digits_y, palette='Set2')

kmeans_pca = KMeans(n_clusters=10, random_state=42).fit(digits_pca)
labels_pca = kmeans_pca.labels_


elm_pca = adjusted_rand_score(digits_y, labels_pca)
print("ELM-metric for PCA-reduced data:", elm_pca)

kmeans_orig = KMeans(n_clusters=10, random_state=42).fit(digits_X)
labels_orig = kmeans_orig.labels_
elm_orig = adjusted_rand_score(digits_y, labels_orig)
print("ELM-metric for original data:", elm_orig)

if elm_pca > elm_orig:
    print("K-means op PCA-reduced data is beter.")
else:
    print("K-means op origineel data is beter.")




In [ ]:
assert_equal(perform_pca(digits_X).shape[-1], 2)

In [ ]:
# Implement your plotting here
import matplotlib.pyplot as plt
#WRITE YOUR CODE HERE

Does your plot make sense? Do you see similar overlaps as in the book?

YOUR ANSWER HERE

In [ ]:
# Compare K means on both the reduced and unreduced data and evaluation using ELM.
from sklearn.cluster import KMeans

#WRITE YOUR CODE HERE